# Primary Key

A primary key is a column or a group of columns used to uniquely identify a row in a table. 
* The column that participates in the primary key is known as the primary key column.
* A table can have zero or one primary key.
* It cannot have more than one primary key.

It is a good practice to add a primary key to every table. When you add a primary key to a table, PostgreSQL creates a **unique B-tree index** on the column or a group of columns used to define the primary key.

Technically, a primary key constraint is the combination of a `NOT NULL` constraint and a `UNIQUE` constraint.
```postgresql
CREATE TABLE table_name (
  column_1 data_type PRIMARY KEY,
  column_2 data_type,
  …
);
```

**Creating a table with a primary key that consists of two columns**

If the primary key consists of more than one column, you can define it using the table constraint:
```postgresql
CREATE TABLE table_name (
  column_1 data_type,
  column_2 data_type,
  column_3 data_type,
  …
  PRIMARY KEY(column_1, column2, ...)
);
```

**Adding a primary key to an existing table**

To add a primary key to an existing table, you use the `ALTER TABLE ... ADD PRIMARY KEY` statement:
```postgresql
ALTER TABLE table_name
ADD PRIMARY KEY (column_1, column_2, ...);
```

**Assign Constraint name**
* If you don’t explicitly specify the name for the primary key constraint, PostgreSQL will assign a default name to the primary key constraint.
* By default, PostgreSQL uses the format **table-name_pkey** as the default name for the primary key constraint.
* To assign a name for the primary key, you can use the `CONSTRAINT` clause as follows:
```postgresql
CONSTRAINT constraint_name
PRIMARY KEY(column_1, column_2,...);
``

**Adding an auto-incremented primary key to an existing table**
```postgresql
ALTER TABLE vendors
ADD COLUMN vendor_id SERIAL PRIMARY KEY;
```

**Drop a primary key**
```postgresql
ALTER TABLE table_name
DROP CONSTRAINT primary_key_constraint; 
```

# Foreign Key Constraint

## Introduction to PostgreSQL Foreign Key Constraint

In PostgreSQL, a **foreign key** is a column or a group of columns in a table that **uniquely identifies a row in another table**.

A foreign key establishes a link between the data in two tables by referencing the **primary key** or a **unique constraint** of the referenced table.
* The table containing a foreign key is referred to as the **referencing table** or **child table**. 
* Conversely, the table referenced by a foreign key is known as the **referenced table** or **parent table**.

The main purpose of foreign keys is to **maintain referential integrity** in a relational database, ensuring that relationships between the parent and child tables are valid.
* For example, a foreign key **prevents the insertion of values** that do not have corresponding values in the **referenced table**.
* Additionally, a foreign key **maintains consistency** by automatically updating or deleting related rows in the child table when changes occur in the parent table.

A table can have multiple foreign keys depending on its relationships with other tables.

## PostgreSQL foreign key constraint syntax

```postgresql
[CONSTRAINT fk_name]
   FOREIGN KEY(fk_columns)
   REFERENCES parent_table(parent_key_columns)
   [ON DELETE delete_action]
   [ON UPDATE update_action]
```
* First, specify the name for the foreign key constraint after the `CONSTRAINT` keyword.
* The `CONSTRAINT` clause is optional. If you omit it, PostgreSQL will assign an auto-generated name.
* Second, specify one or more foreign key columns in parentheses after the `FOREIGN KEY` keywords.
* Third, specify the parent table and parent key columns referenced by the foreign key columns in the `REFERENCES` clause.
* Finally, specify the desired delete and update actions in the `ON DELETE` and `ON UPDATE` clauses.
* Note that if the foreign key constraint does not have the `ON DELETE` and `ON UPDATE` action, they default to `NO ACTION`.

The delete and update actions determine the behaviors when the primary key in the parent table is deleted and updated.
* Since the primary key is rarely updated, the `ON UPDATE` action is infrequently used in practice.
* We'll focus on the `ON DELETE` action.

PostgreSQL supports the following actions:
* SET NULL
* SET DEFAULT
* RESTRICT
* NO ACTION
* CASCADE

## Foreign Key Example: NO ACTION

```postgresql
DROP TABLE IF EXISTS customers;
DROP TABLE IF EXISTS contacts;

CREATE TABLE customers(
   customer_id INT GENERATED ALWAYS AS IDENTITY,
   customer_name VARCHAR(255) NOT NULL,
   PRIMARY KEY(customer_id)
);

CREATE TABLE contacts(
   contact_id INT GENERATED ALWAYS AS IDENTITY,
   customer_id INT,
   contact_name VARCHAR(255) NOT NULL,
   phone VARCHAR(15),
   email VARCHAR(100),
   PRIMARY KEY(contact_id),
   CONSTRAINT fk_customer
      FOREIGN KEY(customer_id)
        REFERENCES customers(customer_id)
); 
```

**Inserts data into the `customers` and `contacts` tables**:
```postgresql
INSERT INTO customers(customer_name)
VALUES('BlueBird Inc'),
      ('Dolphin LLC');

INSERT INTO contacts(customer_id, contact_name, phone, email)
VALUES(1,'John Doe','(408)-111-1234','john.doe@example.com'),
      (1,'Jane Doe','(408)-111-1235','jane.doe@example.com'),
      (2,'David Wright','(408)-222-1234','david.wright@example.com'); 
```

**Delete the customer with ID 1 from the `customers` table**:
```postgresql
DELETE FROM customers
WHERE customer_id = 1;
```

Because of the `ON DELETE NO ACTION`, PostgreSQL issues a constraint violation because the referencing rows of the customer ID 1 still exist in the `contacts` table:
```postgresql
ERROR:  update or delete on table "customers" violates foreign key constraint "fk_customer" on table "contacts".
DETAIL:  Key (customer_id)=(1) is still referenced from table "contacts".
SQL state: 23503
```

* The `RESTRICT` action is similar to the `NO ACTION`. 
* The difference only arises when you define the foreign key constraint as `DEFERRABLE` with an `INITIALLY DEFERRED` or `INITIALLY IMMEDIATE` mode.


## Foreign Key Example: SET NULL

The `SET NULL` automatically sets `NULL` to the foreign key columns in the referencing rows of the child table when the referenced rows in the parent table are deleted.

```postgresql
DROP TABLE IF EXISTS contacts;
DROP TABLE IF EXISTS customers;

CREATE TABLE customers(
   customer_id INT GENERATED ALWAYS AS IDENTITY,
   customer_name VARCHAR(255) NOT NULL,
   PRIMARY KEY(customer_id)
);

CREATE TABLE contacts(
   contact_id INT GENERATED ALWAYS AS IDENTITY,
   customer_id INT,
   contact_name VARCHAR(255) NOT NULL,
   phone VARCHAR(15),
   email VARCHAR(100),
   PRIMARY KEY(contact_id),
   CONSTRAINT fk_customer
      FOREIGN KEY(customer_id)
	  REFERENCES customers(customer_id)
	  ON DELETE SET NULL
);
```

**Insert data into the `customers` and `contacts` tables**:
```postgresql
INSERT INTO customers(customer_name)
VALUES('BlueBird Inc'),
      ('Dolphin LLC');

INSERT INTO contacts(customer_id, contact_name, phone, email)
VALUES(1,'John Doe','(408)-111-1234','john.doe@example.com'),
      (1,'Jane Doe','(408)-111-1235','jane.doe@example.com'),
      (2,'David Wright','(408)-222-1234','david.wright@example.com');
```

**Delete the customer with id 1 from the `customers` table:**
```postgresql
DELETE FROM customers
WHERE customer_id = 1;
```

Because of the `ON DELETE SET NULL` action, the referencing rows in the contacts table are set to `NULL`.

## Foreign Key Example: CASCADE

* The `ON DELETE CASCADE` automatically deletes all the referencing rows in the child table when the referenced rows in the parent table are deleted.
* In practice, the `ON DELETE CASCADE` is the most commonly used option.
```postgresql
DROP TABLE IF EXISTS contacts;
DROP TABLE IF EXISTS customers;

CREATE TABLE customers(
   customer_id INT GENERATED ALWAYS AS IDENTITY,
   customer_name VARCHAR(255) NOT NULL,
   PRIMARY KEY(customer_id)
);

CREATE TABLE contacts(
   contact_id INT GENERATED ALWAYS AS IDENTITY,
   customer_id INT,
   contact_name VARCHAR(255) NOT NULL,
   phone VARCHAR(15),
   email VARCHAR(100),
   PRIMARY KEY(contact_id),
   CONSTRAINT fk_customer
      FOREIGN KEY(customer_id)
	  REFERENCES customers(customer_id)
	  ON DELETE CASCADE
); 
```

**Insert data into the `customers` and `contacts` tables:**
```postgresql
INSERT INTO customers(customer_name)
VALUES('BlueBird Inc'),
      ('Dolphin LLC');

INSERT INTO contacts(customer_id, contact_name, phone, email)
VALUES(1,'John Doe','(408)-111-1234','john.doe@example.com'),
      (1,'Jane Doe','(408)-111-1235','jane.doe@example.com'),
      (2,'David Wright','(408)-222-1234','david.wright@example.com');
```

**Deletes the customer ID 1**:
```postgresql
DELETE FROM customers
WHERE customer_id = 1;
```

Because of the `ON DELETE CASCADE` action, all the referencing rows in the contacts table are automatically deleted.

## Foreign Key Example: SET DEFAULT

The `ON DELETE SET DEFAULT` sets the default value to the foreign key column of the referencing rows in the child table when the referenced rows from the parent table are deleted.

## Add a foreign key constraint to an existing table

```postgresql
ALTER TABLE child_table
ADD CONSTRAINT constraint_name
FOREIGN KEY (fk_columns)
REFERENCES parent_table (parent_key_columns);
```

When adding a foreign key constraint with the `ON DELETE CASCADE` option to an existing table, you need to follow these steps:

First, drop the existing foreign key constraint:
```postgresql
ALTER TABLE child_table
DROP CONSTRAINT constraint_fkey;
```

Second, add a new foreign key constraint with `ON DELETE CASCADE` action:
```postgresql
ALTER TABLE child_table
ADD CONSTRAINT constraint_fk
FOREIGN KEY (fk_columns)
REFERENCES parent_table(parent_key_columns)
ON DELETE CASCADE;
```